MLP对数据进行分类

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim
import pandas as pd
import numpy as np
from sklearn import datasets
from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

定义网络

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(28, 50)
        self.fc2 = nn.Linear(50, 50)
        self.fc3 = nn.Linear(50, 50)
        self.fc4 = nn.Linear(50, 50)
        self.fc5 = nn.Linear(50, 50)
        self.fc6 = nn.Linear(50, 50)
        self.fc7 = nn.Linear(50, 1)

    def forward(self, x):
        out = self.fc1(x)
        out = F.relu(out)
        out = self.fc2(out)
        out = F.relu(out)
        out = self.fc3(out)
        out = F.relu(out)
        out = self.fc4(out)
        out = F.relu(out)
        out = self.fc5(out)
        out = F.relu(out)
        out = self.fc6(out)
        out = F.relu(out)
        out = self.fc7(out)
        return out


def test(predict, label):
    t = predict == label
    accu = np.mean(t)
    return accu

读取数据

In [3]:
table = pd.read_csv(r'D:\repos\py\temp\online_shoppers_intention.csv')
table = shuffle(table, random_state=3)
y = table.iloc[:, -1].apply(lambda x: 1 if x is True else 0)
table = table.drop(columns=['Revenue'])
X = pd.get_dummies(table)
mm = MinMaxScaler(feature_range=(0, 1))
X = mm.fit_transform(X)
train_data, test_data, train_label, test_label = train_test_split(X, y, test_size=0.3, random_state=3)
test_label = np.array(test_label).ravel()
train_label = np.array(train_label).ravel()

定义测试函数

In [4]:
def check(epoch):
    with torch.no_grad():
        test_in = torch.from_numpy(test_data).float()
        test_out = myNet(test_in).squeeze().numpy()
        for i in range(len(test_out)):
            if test_out[i] < 0.5:
                test_out[i] = 0
            else:
                test_out[i] = 1
        accu = test(test_out, test_label)
        f1 = f1_score(test_label, test_out)
        print("Epoch:{}\tLoss:{:.10f}\tAccuracy:{:.10f}\tF1-score:{:.10f}".format(epoch + 1, loss.item(), accu, f1))

In [5]:
myNet=Net()
criterion = nn.MSELoss()  # 损失函数
optimzer = torch.optim.ASGD(myNet.parameters(), lr=0.15)
epochs = 10000  # 训练次数

for i in range(epochs):
    x = torch.from_numpy(train_data).float()
    y = torch.from_numpy(train_label).float()
    out = myNet(x).squeeze()
    loss = criterion(out, y)
    optimzer.zero_grad()
    loss.backward()
    optimzer.step()
    if (i+1) % 500 == 0:  # 每100次输出相关的信息
        check(i)

Epoch:500	Loss:0.1307851076	Accuracy:0.8453636118	F1-score:0.0000000000
Epoch:1000	Loss:0.1307154596	Accuracy:0.8453636118	F1-score:0.0000000000
Epoch:1500	Loss:0.1305194497	Accuracy:0.8453636118	F1-score:0.0000000000
Epoch:2000	Loss:0.1292440295	Accuracy:0.8453636118	F1-score:0.0000000000
Epoch:2500	Loss:0.1193707958	Accuracy:0.8453636118	F1-score:0.0000000000
Epoch:3000	Loss:0.1101081967	Accuracy:0.8769937821	F1-score:0.4678362573
Epoch:3500	Loss:0.0878900588	Accuracy:0.8775344688	F1-score:0.3968042610
Epoch:4000	Loss:0.0898893327	Accuracy:0.8688834820	F1-score:0.2981186686
Epoch:4500	Loss:0.0782355294	Accuracy:0.8961881590	F1-score:0.6577540107
Epoch:5000	Loss:0.0983531997	Accuracy:0.8969991890	F1-score:0.6201395813
Epoch:5500	Loss:0.0767171830	Accuracy:0.8940254123	F1-score:0.5838641189
Epoch:6000	Loss:0.0745327845	Accuracy:0.8961881590	F1-score:0.6660869565
Epoch:6500	Loss:0.0740226433	Accuracy:0.8964585023	F1-score:0.6039296794
Epoch:7000	Loss:0.0743650347	Accuracy:0.8926736956	F